# Graboid documentation
## Calibration
This module performs the leave-one-out validation over different combinations of parameters explored trough a grid search

### Functions
* **get_metrics(confusion, taxons)** REturns the *accuracy*, *precision*, *recall* and *F1 score* for each taxon in *taxons*
* **get_report_filename(out_tab, mat_path)**
* **loo_generator(nrecs)** Yields the indexes for the training dataset and the testing instance in leave-one-out calibration
* **build_confusion(pred, real)** Build a confusion matrix from the given predicted (*pred*) and real (*real*) values. Returns the generated confusion matrix and an array containing the TaxIDs of the represented taxons
* **build_cal_tab(pred_tax, real_tax, n_ranks = 6)** Builds a calibration result table from the given predicted values (*pred_tax*) and real values (*real_tax*). Returns array with columns *rank*, *taxon*, *accuracy*, *precision*, *recall*, *F1 score*
* **loo_calibrate(garrus, w_size, w_step, max_k, step_k, max_sites, step_sites, dist_mat)**

### Calibrator
**class Calibrator(out_dir, tmp_dir, warn_dir)**
<ins>NOTE: currently named *Calibrator0*, replace *Calibrator* after testing</ins>

This class handles the grid search for a range of different parameters. Stores the result files. Allow resetting of taxonomic rank and distance matrix used in feature selection and cluster data calculation

#### Parameters
* **out_dir**
* **tmp_dir**
* **warn_dir**

#### Attributes
* **out_dir** Target directory for the generated reports
* **warn_dir** Target directory for the generated warning files
* **log_handler** Handler for the generated logs
* **warn_handler** Handler for the generated warnings
* **loader** *WindowLoader* instance used to retrieve the appropriate windows in each step of the calibration
* **row_thresh** Threshold proportion of empty rows to filter. Default: 0.2
* **col_thresh** Threshold proportion of empty columns to filter. Default: 0.2
* **min_seqs** Minimum sequences to admit a taxon. Default: 10
* **rank** Rank to be used in the feature selection step. Default: 'genus'
* **cost_mat** Cost matrix to be used in distance calculation. Default: K2P matrix with transitions = 1 and transversions = 2
* **report** Dataframe with the results of the last calibration round

#### Methods
* **set_database(mat_file, acc_file, tax_file)** Instantiate the *loader* object, passing *mat_file*, *acc_file*, *tax_file* as input files for the sequence data, accession list and taxonomy table respectively
* **set_row_thresh(thresh = 0.2)** Reset *row_thresh*
* **set_col_thresh(thresh = 0.2)** Reset *col_thresh*
* **set_min_seqs(thresh = 0.2)** Reset *min_seqs*
* **set_rank(rank = 'genus')** Reset *rank*
* **set_cost_mat(transition = 1, transversion = 2, id = False)** Reset *cost_mat* with the given values for *transition* and *transversion*. If *id* is True, use an identity matrix instead
* **grid_search(w_size, w_step, max_k, step_k, max_n, step_n, min_k = 1, min_n = 5)** Perform a grid search for the given parameter ranges. Arguments *w_size* and *w_step* determine the length and displacement speed of the sliding window. Arguments *max_k* and *step_k* determine the range of *k*, optional argument *min_k* sets the lower bound to the range. Arguments *max_n* and *step_n* determine the range of *n*, optional argument *min_n* sets the lower bound to the range. If The last sliding window overshoots the alignment length, replace it with a tail window, starting *w_size* places before the alignment end. For each window perform leave-one-out calibration for every combination of *k* and *n*, using the three classification modes (*majority*, *wknn*, *dwknn*). Generate a dataframe with columns *rank*, *taxon*, *accuracy*, *precision*, *recall*, *F1_score*, *w_start*, *w_end*, *K*, *n_sites*, and *mode*. Store it to *report*
* **save_report(filename)** Save the last generated report to the given *filename* (in *out_dir*)

### Rerporter
This module handles visualization of the calibration report. Used in experiment design and query classification

#### Functions
* **build_plot_tab(report_tab, metric = 'F1_score')** Prepares the report table to build the report heatmap. Builds dataframes *met_tab* and *param_tab*. *met_tab* contains the best *metric* value for each taxon for each window (windows in which the taxon is not represented take value -1). *param_tab* contains the combination of *K*, *n* and *mode* that yielded the best *metric* value for each taxon for each window.
* **plot_report(report_tab, tax_dict = None, metric = 'F1_score', rank = None)** Constructs a heatmap for the given *report_tab*, with rows = taxons and columns = windows. The color of each cell shows the best *metric* value for its taxon/window, and inside the square is shown the parameter combination that yelded the value. If *tax_dict* is given, use it to replace the taxa numeric IDs for their taxon names

#### ReportLoader
**class ReportLoader()**

This class loads a calibration report and extracts parameter combinations with the best metrics.

##### Attributes
* **report_file** Path to the file containing the calibration report
* **report** Calibration report table
* **taxguide_file** Path to the file containing the TaxID guide table
* **taxguide** TaxID guide table
* **tax_dict** Dictionary containing *taxid*:*taxon* key:values

##### Methods
* **get_report(report_file)** Load *report_file*
* **get_taxguide(taxguide_file)** Load *taxguide_file*
* **query_tax(\*taxa, w_start = None, w_end = None, metric = 'F1_score')** Locate the given *taxa* in the report table. If *w_start* and *w_end* are given, report result bounded by those positions. For each classification method, return the best result for the given *metric*. Return a sub table of the report containing the rows corresponding to the parameter combination that yields the best *metric*
* **query_window(w_start = 0, w_end = inf, metric = 'F1_score')** Locate the windows bounded by positions *w_start* and *w_end*. From each taxon located within these confines, return the parameter combinations that yield the best *metric*

#### Director
**class Director(out_dir)**

This class is used to facilitate accessing and plotting specific portions of the calibration report

##### Attributes
* **out_dir** Output directory for the generated files
* **loader** *ReportLoader* instance, used to access the calibration report
* **report** Cropped report, containing the queried *taxa* and/or *windows*

##### Methods
* **set_data(report_file, taxguide_file)** Select the files to be used. *report_file* contains the calibration report, *taxguide_file* contains the taxonomic ID guide for the represented taxa
* **query_report(metric = 'F1_score')** Get the best *metric* for each taxa in each window, along with the parameter combination that yielded it
* **query_window(w_start, w_end, metric = 'F1_score')** Get the best *metric* for each taxa in the space delimited by *w_start* and *w_end*, along with the parameter combination that yielded it
* **query_tax(\*taxa, w_start = None, w_end = None, metric = 'F1_score')** Get the best *metric* for the specified *taxa*, along with the parameter combination that yielded it. If *w_start* and *w_end* are given, bound the search within those positions
* **plot_report(metric = 'F1_score', rank = None)** Builds a heatmap for the last generated report, using the specified *metric*. If a taxonomic rank is given, use only taxa belonging to said *rank*
* **save_report(out_file)** Save the last generated report to the given *out_file* (in the directory *out_dir*)